# PyLossless pipeline: A step-by-step demonstration

This Notebook details each pipeline step. First, the mathematical notation for the operation will be described. Second, the python code for the operation will be shown. We will go through the first step in details. Similar procedures are used in all following steps so for the sake of brevity we will increasingly explain less of the formulas throughout this tutorial

# ----------------------------------------------------------------------------

# Cloning and installing pylossless

In [1]:
# THIS CODE ONLY NEEDS TO BE RUN ONCE.
# IF PYLOSSLESS IS ALREADY AVAILABLE, IT DOES NOT NEED TO BE RERUN.
try:
  import pylossless
except ModuleNotFoundError:
  !git clone https://github.com/lina-usc/pylossless.git
  %pip install --quiet --editable ./pylossless

  # RESTARTING THE KERNEL AFTER THE PIP INSTALL
  # THIS IS NECESSARY FOR THE NEWLY INSTALLED PACKAGE
  # TO BE CORRECTLY IMPORTED.
  exit()


<br>

--------------------------------
# Getting a LosslessPipeline object

 Create a pipeline object and manually set a raw object to it. We proceed this way to be able to demonstrate the operation for each step, one at a time. Normally, one would simply load and process a raw object with `pipeline.run_with_raw(raw)`. This demonstration relies on test datasets from MNE. It may take a while to download the MNE sample data (about 2-3 minutes on Google Colab). **Please be patient**...

In [2]:
from pathlib import Path

import numpy as np

import mne
from mne.datasets import sample

import pylossless as ll

# LOAD DATA FROM MNE FOR THE DEMONSTRATION
data_path = sample.data_path()
meg_path = data_path / 'MEG' / 'sample'
raw_fname = meg_path / 'sample_audvis_raw.fif'
raw = mne.io.read_raw_fif(raw_fname, preload=True)

# LOAD DEFAULT CONFIG
config = ll.config.Config()
config.load_default()
config.save("test_config.yaml")

# CREATE A PIPELINE AN MANUALLY SET A RAW OBJECT
pipeline = ll.LosslessPipeline('test_config.yaml')
pipeline.raw = raw

Opening raw data file /Users/scotterik/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...


--------------------------

# Preamble: Notation

We define:
- s, e, and t, as sensor, epochs, and time dimensions, respectively.

- a 3D matrix $X \in \mathbb{R}^{S_\mathcal{G} \times E_\mathcal{G} \times T}$, where $S_\mathcal{G}$  and $E_\mathcal{G}$ are the set of good sensors and epochs respectively, and $T$ is the number of time points (i.e. samples)

- We use superscript to denote operations across a dimension, and we use subscript to denote indexing a dimension


- a single sensor $i$ as $ X\big|_{s=i} \in \mathbb{R}^{E_\mathcal{G} \times T}$, with $i \in S_\mathcal{G}$

- a single epoch $j$ as  $X\big|_{e=j} \in \mathbb{R}^{S_\mathcal{G} \times T}$, with $j \in E_\mathcal{G} $

- sensor specific thresholds for rejecting epochs as $\tau^e_i \in \mathbb{R}^{S_\mathcal{G}}$

- epoch specific thresholds for rejecting sensors as $\tau^s_j \in \mathbb{R}^{E_\mathcal{G}}$


- *quantiles* as $Q\#^{dim}$: i.e. $Q75^s$ is the 75th *quantile* along the sensor dimension. The function $Q75^s(X)$ computes the 75th quantile along the $s$ dimension of matrix $X$, resulting in a matrix noted $X^{Q75^s} \in \mathbb{R}^{E \times T}$

Throughout the text, we use capital letters for matrices and lower-case letters for scalars. For example, the data point for sensor $i$, epoch $j$ and time $k$ as $X\big|_{s=i; e=j; t=k} = x_{ijk}\in \mathbb{R}$, and $X=\{x_{ijk}\}$.

<br>

------------------------

# Step 0: Input Data

### First, we epoch the data to be used for step 1

Let our 3D matrix below be defined as $X \in \mathbb{R}^{S \times E \times T}$ where $X$ is a matrix of real numbers and of dimension $S$ sensors $\times$ $E$ epochs $\times$ $T$ times.

In [3]:
epochs = pipeline.get_epochs()

🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 601 original time points ...
0 bad epochs dropped
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


In [4]:
# Let us convert our epochs array into a Named Array
from pylossless.pipeline import epochs_to_xr
epochs_xr = epochs_to_xr(epochs, kind='ch')
epochs_xr # 277 epochs, 50 sensors, 602 samples per epoch

<xarray.DataArray (epoch: 277, ch: 59, time: 601)> Size: 79MB
array([[[-4.26965878e-05, -4.55733025e-05, -4.78312309e-05, ...,
         -6.90473304e-05, -8.13633825e-05, -7.19575301e-05],
        [-1.12694344e-05, -1.20099186e-05, -1.16296023e-05, ...,
         -1.13520186e-05, -1.14285380e-05, -1.30872915e-05],
        [-1.78352282e-05, -2.42311691e-05, -2.27521589e-05, ...,
         -1.45089987e-05, -9.11264817e-06, -5.92725481e-06],
        ...,
        [ 1.50136773e-05,  1.54548337e-05,  1.51369078e-05, ...,
          2.23180254e-05,  2.22599691e-05,  2.24156382e-05],
        [-1.52412965e-05, -1.46724240e-05, -1.45173275e-05, ...,
         -9.34228283e-06, -9.62922101e-06, -9.64063562e-06],
        [ 1.17435988e-05,  1.25791034e-05,  1.26648640e-05, ...,
          1.96664164e-05,  1.92420881e-05,  1.86432406e-05]],

       [[-7.19575301e-05, -5.53443722e-05, -4.86540536e-05, ...,
         -5.51620239e-05, -5.48918032e-05, -5.74360574e-05],
        [-1.30872915e-05, -1.57002080e-05, -1.71937885e-05, ...,
         -2.50080635e-05, -2.33215219e-05, -2.28248519e-05],
        [-5.92725481e-06, -5.81562304e-06, -6.09936458e-06, ...,
         -2.19680530e-05, -2.03989737e-05, -1.92719632e-05],
...
          2.54402386e-05,  2.49608841e-05,  2.48504591e-05],
        [-1.87772370e-05, -1.81752141e-05, -1.82906611e-05, ...,
         -2.31687121e-06, -3.40169454e-06, -3.79578042e-06],
        [ 4.34385855e-06,  5.55232672e-06,  5.85976184e-06, ...,
          1.88530226e-05,  1.83717758e-05,  1.94134504e-05]],

       [[-3.84674103e-05, -3.62507308e-05, -3.51405898e-05, ...,
         -6.79203142e-05, -6.71273687e-05, -6.00922247e-05],
        [-1.70288598e-06,  2.03242703e-08,  1.64636495e-06, ...,
         -8.99987535e-06, -9.68092889e-06, -1.11533229e-05],
        [-8.89306783e-06, -7.38259569e-06, -4.71025977e-06, ...,
         -1.77159054e-05, -1.77999741e-05, -1.83879992e-05],
        ...,
        [ 2.48504591e-05,  2.47239989e-05,  2.32465546e-05, ...,
          2.23522063e-05,  2.20773417e-05,  2.27953468e-05],
        [-3.79578042e-06, -3.83465514e-06, -5.35863884e-06, ...,
         -5.48836914e-06, -4.95543344e-06, -4.01205597e-06],
        [ 1.94134504e-05,  2.12006254e-05,  2.04842284e-05, ...,
          8.82419425e-06,  8.55354110e-06,  8.66345301e-06]]],
      shape=(277, 59, 601))
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'
  * time     (time) float64 5kB 0.0 0.001665 0.00333 ... 0.9956 0.9973 0.999

-------------------------

# Step 1: Flag Noisy Sensors

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_noisy_sensors.png" alt="Flag noisy channels" width="50%" height="50%">


## step 1a) Take standard deviation across temporal dimension

#### We take the standard deviation of $X \in \mathbb{R}^{S \times E \times T}$ across the samples dimension time, resulting in a 2D matrix $X^{\sigma_{t}} \in \mathbb{R}^{S \times E}$


![std across samples](https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/samples_std.png)


<br>

---------------------------------


In [5]:
# Take standard deviation across time dimension
trim_ch_sd = epochs_xr.std('time')
# display a small subset
trim_ch_sd

<xarray.DataArray (epoch: 277, ch: 59)> Size: 131kB
array([[2.09444784e-05, 2.07527689e-05, 2.38033814e-05, ...,
        1.02160602e-05, 1.03841106e-05, 1.04743124e-05],
       [9.05919749e-06, 5.93565086e-06, 6.40798303e-06, ...,
        3.96903967e-06, 4.06371123e-06, 5.16576878e-06],
       [1.69014959e-05, 1.71864979e-05, 1.99197789e-05, ...,
        6.62919877e-06, 7.21621460e-06, 6.89065680e-06],
       ...,
       [7.54258545e-06, 4.88883244e-06, 5.60358362e-06, ...,
        4.30815311e-06, 4.89011551e-06, 4.85976514e-06],
       [7.13885371e-06, 5.97633007e-06, 5.68398848e-06, ...,
        5.04293509e-06, 6.60209421e-06, 5.44856698e-06],
       [6.83438970e-06, 5.42294775e-06, 7.71352335e-06, ...,
        5.68786404e-06, 5.91042479e-06, 6.40245612e-06]], shape=(277, 59))
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

------------------

## Take the 50th and 75th quantile across dimension sensor of $X^{\sigma_{t}}$


This operation results in two 1D vectors of size $E$:

- $ X^{{\sigma}_t{Q50^s}} = Q50^s(X^{\sigma_{t}}) \in \mathbb{R}^{E} $
- $ X^{{\sigma}_t{Q75^s}} = Q75^s(X^{\sigma_{t}}) \in \mathbb{R}^{E} $



In [6]:
# Step a
q50, q75 = trim_ch_sd.quantile([.5, .75], dim='ch')
q50 # a 1D array of median stDev values across channels for each epoch

<xarray.DataArray (epoch: 277)> Size: 2kB
array([7.78990634e-06, 4.63682013e-06, 5.52568767e-06, 5.93209087e-06,
       4.50753766e-06, 5.07751977e-06, 5.31909744e-06, 4.00034339e-06,
       4.88190530e-06, 4.38886763e-06, 3.97889933e-06, 4.94461557e-06,
       5.29868656e-06, 4.45642559e-06, 9.48557029e-06, 5.00245456e-06,
       5.82777343e-06, 4.31875959e-06, 5.00821415e-06, 5.13644420e-06,
       5.34530002e-06, 4.99159866e-06, 5.69866846e-06, 5.64267865e-06,
       4.96441489e-06, 5.16164771e-06, 5.61352429e-06, 5.60552758e-06,
       5.14768383e-06, 4.66683924e-06, 7.69332083e-06, 6.09805410e-06,
       8.19737128e-06, 5.20358998e-06, 4.94967074e-06, 5.05406491e-06,
       4.26506325e-06, 4.62337499e-06, 4.51914913e-06, 5.05335183e-06,
       5.02806020e-06, 4.74145240e-06, 5.45198489e-06, 8.38736612e-06,
       5.84296674e-06, 9.55729903e-06, 6.68646280e-06, 8.82973023e-06,
       7.14895224e-06, 5.94684544e-06, 4.80543643e-06, 5.96801410e-06,
       5.82451170e-06, 4.63533065e-06, 4.32163191e-06, 5.23137281e-06,
       6.80396354e-06, 7.72940600e-06, 5.48140882e-06, 8.47552938e-06,
       1.00543777e-05, 6.38478032e-06, 6.51461646e-06, 5.60788253e-06,
       5.21652300e-06, 5.94449098e-06, 6.48727894e-06, 4.32178800e-06,
       4.50359096e-06, 6.83187955e-06, 7.08057309e-06, 7.01197406e-06,
       7.44312454e-06, 6.70128951e-06, 7.14703936e-06, 6.35876077e-06,
       5.94828758e-06, 5.15399905e-06, 5.14324294e-06, 5.65800598e-06,
...
       5.48843041e-06, 4.48311477e-06, 6.23545382e-06, 6.33879598e-06,
       5.05426582e-06, 5.28134403e-06, 5.34771075e-06, 6.24743705e-06,
       6.46077687e-06, 4.90228189e-06, 5.86282846e-06, 6.69697187e-06,
       4.72779680e-06, 4.46928489e-06, 4.47456299e-06, 4.88179239e-06,
       6.37953137e-06, 6.15119534e-06, 8.71713638e-06, 1.21473462e-05,
       9.11572869e-06, 7.97947213e-06, 9.01891118e-06, 8.39687401e-06,
       5.66294282e-06, 5.52082022e-06, 5.89181023e-06, 6.82930211e-06,
       5.40630373e-06, 5.17800732e-06, 4.45832887e-06, 6.89429939e-06,
       7.32057710e-06, 7.10780210e-06, 5.32810710e-06, 5.42592627e-06,
       5.01458318e-06, 5.98140021e-06, 5.51320290e-06, 4.64460858e-06,
       7.01127444e-06, 7.74087896e-06, 8.50321620e-06, 8.64863602e-06,
       6.30043802e-06, 4.86382616e-06, 5.61462558e-06, 9.38739685e-06,
       7.25777363e-06, 9.42268962e-06, 9.66169784e-06, 8.03482724e-06,
       8.19420331e-06, 9.00539065e-06, 7.94254148e-06, 7.24636215e-06,
       6.97503220e-06, 8.37327893e-06, 8.32851420e-06, 6.96840358e-06,
       6.07869048e-06, 5.43649315e-06, 5.39901876e-06, 5.15162545e-06,
       5.48920192e-06, 6.45963996e-06, 5.22415315e-06, 8.33443445e-06,
       6.33564992e-06, 7.31244342e-06, 1.19384389e-05, 8.84563437e-06,
       7.60488407e-06, 5.82795246e-06, 4.88883244e-06, 5.44856698e-06,
       6.08648731e-06])
Coordinates:
  * epoch     (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276
    quantile  float64 8B 0.5

## 1b) Let the *Upper Quantile Range* be defined by $Q75 - Q50$
$ UQR^s = X^{{\sigma}_T{Q75}^s} - X^{{\sigma}_T{Q50}^s}$

In [7]:
# step B. Define the threshold
uqr = q75 - q50
uqr  # Q70 - Q50

<xarray.DataArray (epoch: 277)> Size: 2kB
array([1.65457085e-06, 1.27995053e-06, 1.03766141e-06, 1.25358195e-06,
       1.29937462e-06, 1.13096659e-06, 1.09989401e-06, 1.71082625e-06,
       1.44885966e-06, 1.54382443e-06, 1.51169177e-06, 1.04487878e-06,
       1.30142777e-06, 1.34326180e-06, 3.13723400e-06, 8.34921857e-07,
       1.24881450e-06, 1.03609640e-06, 8.94155647e-07, 1.29817008e-06,
       6.53519965e-07, 1.48675690e-06, 1.01166706e-06, 1.37342657e-06,
       1.48205582e-06, 1.25230582e-06, 1.34104535e-06, 9.47906391e-07,
       1.62455406e-06, 1.18145545e-06, 2.02406408e-06, 1.32170954e-06,
       1.42636168e-06, 1.23180861e-06, 1.05628122e-06, 1.50697819e-06,
       1.42955946e-06, 1.63494371e-06, 1.42661465e-06, 1.31619429e-06,
       1.14902636e-06, 1.78641409e-06, 1.09589200e-06, 2.04961574e-06,
       9.25318307e-07, 1.85482186e-06, 9.82294776e-07, 1.14770834e-06,
       1.56632162e-06, 1.41667547e-06, 1.77377252e-06, 1.12545904e-06,
       1.21347161e-06, 1.36055337e-06, 1.83613636e-06, 1.54243680e-06,
       2.62542552e-06, 1.62490501e-06, 2.04438399e-06, 2.09415862e-06,
       1.57586130e-06, 1.17119415e-06, 1.87376295e-06, 1.31702728e-06,
       1.27268629e-06, 1.24418230e-06, 1.19448401e-06, 2.02316259e-06,
       1.87959956e-06, 2.33519892e-06, 2.12552010e-06, 1.27780941e-06,
       1.92730064e-06, 1.25787979e-06, 1.02514705e-06, 1.63343199e-06,
       1.34324651e-06, 1.96646356e-06, 1.59104395e-06, 1.55265503e-06,
...
       2.69307531e-06, 2.94829381e-06, 1.65739512e-06, 1.56032219e-06,
       1.87271352e-06, 2.12178632e-06, 1.70056408e-06, 1.96775671e-06,
       1.55108662e-06, 2.23872594e-06, 1.61030939e-06, 2.34533596e-06,
       2.50905929e-06, 2.23867032e-06, 2.71115906e-06, 2.40587753e-06,
       1.70562306e-06, 1.77345634e-06, 2.47319955e-06, 2.61409163e-06,
       4.74654419e-06, 2.06040739e-06, 1.31249000e-06, 1.45408856e-06,
       1.38274215e-06, 1.59322913e-06, 1.85175083e-06, 1.71929272e-06,
       1.98441908e-06, 1.87615785e-06, 2.14399889e-06, 1.77130282e-06,
       2.12432173e-06, 2.07429743e-06, 1.66370367e-06, 1.80792395e-06,
       2.99896758e-06, 1.74123814e-06, 1.19468245e-06, 1.72332368e-06,
       1.28746674e-06, 8.66862183e-07, 1.15064400e-06, 1.83532465e-06,
       8.97575278e-07, 9.88472296e-07, 1.43746115e-06, 2.26510344e-06,
       1.27376573e-06, 1.01611176e-06, 1.43769296e-06, 1.42905679e-06,
       1.40609152e-06, 1.82515776e-06, 1.30694249e-06, 1.95355184e-06,
       1.01947785e-06, 2.30066014e-06, 8.30370190e-07, 1.16309949e-06,
       8.91354303e-07, 6.55841804e-07, 1.45472028e-06, 2.62044625e-06,
       2.18524805e-06, 1.41987452e-06, 1.74161061e-06, 1.93230978e-06,
       1.45398171e-06, 1.48846169e-06, 3.84976241e-06, 1.64488411e-06,
       9.95601577e-07, 1.19994708e-06, 1.44843903e-06, 1.13196264e-06,
       1.21903321e-06])
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276

---------------

### Identify outlier Indices $(i, j)$

We multiply a constant $k$ by the $UQR$ to define a measure for the spread of the right tail of the distribution of $X^{\sigma_{t}}$ values and add it to the median of $X^{\sigma_{t}}$ to obtain epoch-specific standard deviation threshold for outliers:

$$
    \tau^s_j = X^{{\sigma}_T{Q50}^S} + UQR^s\times k
$$

That is, $\tau^s_j$ is the epoch-specific threshold for the epoch $j$.

In [8]:
# Step 1b part 2
k = 3
u_out =  q50 + q75 * k
u_out  # epoch specific thresholds

<xarray.DataArray (epoch: 277)> Size: 2kB
array([3.61233379e-05, 2.23871321e-05, 2.52157349e-05, 2.74891093e-05,
       2.19282745e-05, 2.37029789e-05, 2.45760718e-05, 2.11338523e-05,
       2.38742002e-05, 2.21869438e-05, 2.04506726e-05, 2.29130986e-05,
       2.50990296e-05, 2.18554878e-05, 4.73539832e-05, 2.25145838e-05,
       2.70575372e-05, 2.03833275e-05, 2.27153235e-05, 2.44402870e-05,
       2.33417600e-05, 2.44266654e-05, 2.58296750e-05, 2.66909943e-05,
       2.43038270e-05, 2.44035083e-05, 2.64772332e-05, 2.52658295e-05,
       2.54643975e-05, 2.22117233e-05, 3.68454756e-05, 2.83573450e-05,
       3.70685702e-05, 2.45097858e-05, 2.29675266e-05, 2.47371942e-05,
       2.13489314e-05, 2.33983311e-05, 2.23564405e-05, 2.41619902e-05,
       2.35593199e-05, 2.43250519e-05, 2.50956156e-05, 3.96983117e-05,
       2.61478219e-05, 4.37936617e-05, 2.96927355e-05, 3.87620459e-05,
       3.32947738e-05, 2.80374082e-05, 2.45430633e-05, 2.72484335e-05,
       2.69384616e-05, 2.26229827e-05, 2.27949367e-05, 2.55528016e-05,
       3.50921307e-05, 3.57923390e-05, 2.80587873e-05, 4.01845934e-05,
       4.49450947e-05, 2.90527037e-05, 3.16797547e-05, 2.63826120e-05,
       2.46841509e-05, 2.75105108e-05, 2.95325678e-05, 2.33566398e-05,
       2.36531625e-05, 3.43331150e-05, 3.46988527e-05, 3.18813245e-05,
       3.55544001e-05, 3.05787974e-05, 3.16635986e-05, 3.03353390e-05,
       2.78228898e-05, 2.65153868e-05, 2.53461036e-05, 2.72899890e-05,
...
       3.00329475e-05, 2.67773405e-05, 2.99140006e-05, 3.00361505e-05,
       2.58352038e-05, 2.74907351e-05, 2.64925353e-05, 3.08930183e-05,
       3.04963673e-05, 2.63253054e-05, 2.82822420e-05, 3.38238954e-05,
       2.64383651e-05, 2.45931505e-05, 2.60317292e-05, 2.67448021e-05,
       3.06349947e-05, 2.99251504e-05, 4.22881442e-05, 5.64316597e-05,
       5.07025473e-05, 3.80991107e-05, 4.00131147e-05, 3.79497617e-05,
       2.67999977e-05, 2.68629683e-05, 2.91224934e-05, 3.24750866e-05,
       2.75784722e-05, 2.63405028e-05, 2.42653121e-05, 3.28911060e-05,
       3.56552736e-05, 3.46541007e-05, 2.63035394e-05, 2.71274769e-05,
       2.90552355e-05, 2.91493153e-05, 2.56368589e-05, 2.37484054e-05,
       3.19074980e-05, 3.35641024e-05, 3.74647968e-05, 4.01005180e-05,
       2.78944779e-05, 2.24207215e-05, 2.67708857e-05, 4.43448977e-05,
       3.28523917e-05, 4.07390938e-05, 4.29598702e-05, 3.64264794e-05,
       3.69950878e-05, 4.14970359e-05, 3.56909934e-05, 3.48461041e-05,
       3.09585623e-05, 4.03950961e-05, 3.58051674e-05, 3.13629128e-05,
       2.69888248e-05, 2.37134980e-05, 2.59602359e-05, 2.84678406e-05,
       2.85125519e-05, 3.00981834e-05, 2.61214444e-05, 3.91346671e-05,
       2.97045448e-05, 3.37151587e-05, 5.93030430e-05, 4.03171898e-05,
       3.34063410e-05, 2.69116511e-05, 2.39006468e-05, 2.51901558e-05,
       2.80030489e-05])
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276

------------

Now, we compare our 2D standard deviation matrix to the threshold vector of $\tau^e_j$:

$$
X^{\sigma_{t}} \big|_{e=j}  > \tau^s_j
$$

resulting in the indicator matrix $C \in \{0, 1\}^{S \times E}=\{c_{ij}\}$ with

$$
c_{ij} =
\begin{cases}
0 & \text{if } x^{\sigma_{t}}_{ij} < \tau^s_j \\
1 & \text{if } x^{\sigma_{t}}_{ij} \geq \tau^s_j
\end{cases}
$$

Each element of this matrix indicates whether sensor $i$ is an outlier at an epoch $j$.


In [9]:
# Step D
outlier_mask = trim_ch_sd > u_out
outlier_mask


<xarray.DataArray (epoch: 277, ch: 59)> Size: 16kB
array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(277, 59))
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

--------------

## 1c) Identify noisy sensors part 1
To identify outlier **sensors**, we average across the **epoch** dimension of our indicator matrix $C$ and obtain $C^{\mu_e} \in \mathbb{R}^{S_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_e}_i$ representing the percentage of **epochs** for which that sensor is an outlier.


In [10]:
prop_outliers = outlier_mask.astype(float).mean('epoch')
prop_outliers # porportion of epochs that sensor is an outlier

<xarray.DataArray (ch: 59)> Size: 472B
array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.00722022, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00361011,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00361011,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.00361011,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ])
Coordinates:
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

## 1d) Identify noisy sensors part 2

Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a sensor should be marked artifactual. The sensor $i$ is flagged as noisy if $c^{\mu_e}_i > \tau^{p}$.


In [11]:
threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED SENSORS

<xarray.DataArray (ch: 0)> Size: 0B
array([], dtype=float64)
Coordinates:
  * ch       (ch) <U7 0B

In [12]:
# Let's add the outlier channels to our flags
bad_chs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED SENSORS
pipeline.flags["ch"].add_flag_cat(kind='ch_sd',
                                  bad_ch_names=bad_chs)
print(pipeline.flags['ch'])

Flagged channels: |
  Noisy: None
  Bridged: None
  Uncorrelated: None
  Rank: None



---------------

# Step 2: Flag Noisy Epochs

#### This step closely resembles the "Flag Noisy Channels" step. For the sake of brevity we will be more concise in the documentation.

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_noisy_epochs.png" alt="Flag noisy epochs" width="50%" height="50%">

--------



----------------------------------------------------------------------------


## step 2a) standard deviation across the samples dimension $T$


### **Take a moment below to notice** that the sensors flagged in step 1 are not in ``epochs_xr``


In [13]:
epochs = pipeline.get_epochs()
# Let's make our epochs array into a named Array
epochs_xr = epochs_to_xr(epochs, kind='ch')
trim_ch_sd = epochs_xr.std("time")
trim_ch_sd

🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 601 original time points ...
0 bad epochs dropped
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.


<xarray.DataArray (epoch: 277, ch: 59)> Size: 131kB
array([[2.09444784e-05, 2.07527689e-05, 2.38033814e-05, ...,
        1.02160602e-05, 1.03841106e-05, 1.04743124e-05],
       [9.05919749e-06, 5.93565086e-06, 6.40798303e-06, ...,
        3.96903967e-06, 4.06371123e-06, 5.16576878e-06],
       [1.69014959e-05, 1.71864979e-05, 1.99197789e-05, ...,
        6.62919877e-06, 7.21621460e-06, 6.89065680e-06],
       ...,
       [7.54258545e-06, 4.88883244e-06, 5.60358362e-06, ...,
        4.30815311e-06, 4.89011551e-06, 4.85976514e-06],
       [7.13885371e-06, 5.97633007e-06, 5.68398848e-06, ...,
        5.04293509e-06, 6.60209421e-06, 5.44856698e-06],
       [6.83438970e-06, 5.42294775e-06, 7.71352335e-06, ...,
        5.68786404e-06, 5.91042479e-06, 6.40245612e-06]], shape=(277, 59))
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

----------

## Step 2a) part 2: We compute $\sigma^{X^{\sigma_{t}}[50,70]e}$




 50th and 75th quantile across epochs and the UQR

<br>

Like before, We Take the median and 70th quantile, but now we operate across **epochs**, resulting in two 1D vector's of size ``n_good_sensors`` $S_\mathcal{G}$

<br>

- $ X^{{\sigma}_t{Q50^e}} = Q50^e(X^{\sigma_{t}}) \in \mathbb{R}^{S_\mathcal{G}} $
- $ X^{{\sigma}_t{Q75^e}} = Q75^e(X^{\sigma_{t}}) \in \mathbb{R}^{S_\mathcal{G}} $

<br>

$ UQR^e = (X^{{\sigma}_T{Q75}^e} - X^{{\sigma}_T{Q50}^e})$

<br>

--------------------------------

## Step 2b: define sensor-specific thresholds for rejecting epochs $\tau^e_i$

Our sensor-specifc threshold for rejecting epochs is defined by:


$ \tau^e_i = X^{{\sigma}_T{Q50}^e} + UQR^e\times k$



In [14]:
# This is step 2a
mid_val, upper_val = trim_ch_sd.quantile([.5, .75], dim='epoch')
upper_dist = upper_val - mid_val
upper_dist

<xarray.DataArray (ch: 59)> Size: 472B
array([3.13291911e-06, 4.07667256e-06, 4.49130019e-06, 1.38334151e-06,
       1.99916437e-06, 2.99639980e-06, 2.83794183e-06, 1.00037780e-06,
       7.88619609e-07, 1.19360397e-06, 1.33749215e-06, 1.49262092e-06,
       1.58098765e-06, 1.51661413e-06, 1.30720056e-06, 1.33796134e-06,
       9.72916371e-07, 8.58177473e-07, 4.55273204e-07, 6.01445491e-07,
       8.42495492e-07, 6.51215106e-07, 6.36336588e-07, 6.90041006e-07,
       1.34747252e-06, 5.43314770e-07, 6.42333359e-07, 7.14036460e-07,
       6.59402375e-07, 8.81704429e-07, 6.81278423e-07, 6.83542142e-07,
       5.88993699e-07, 5.54934182e-07, 1.18817913e-06, 1.30310760e-06,
       1.12561230e-06, 1.16417590e-06, 1.39972190e-06, 1.16843094e-06,
       9.94596279e-07, 1.16693113e-06, 1.41912390e-06, 1.78942291e-06,
       1.62180130e-06, 1.91723580e-06, 1.64789442e-06, 1.94511647e-06,
       1.69169156e-06, 1.33612792e-06, 1.90833139e-06, 1.26799822e-06,
       2.48571382e-06, 1.71734985e-06, 1.60284668e-06, 2.01139436e-06,
       1.89974248e-06, 1.53949332e-06, 1.21562280e-06])
Coordinates:
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

In [15]:
# This is step 2b
k = 3
u_out =  mid_val + upper_dist * k # sensor-specific thresholds

--------

## Step 2c) Identify Noisy Epochs part 1

<br>

#### The indicator matrix is defined by

$$
c_{ij} =
\begin{cases}
0 & \text{if } x^{\sigma_{t}}_{ij} < \tau^e_i \\
1 & \text{if } x^{\sigma_{t}}_{ij} \geq \tau^e_i
\end{cases}
$$

To identify outlier **epochs**, we average across the **sensor** dimension of our indicator matrix $C$ and obtain $C^{\mu_s} \in \mathbb{R}^{E_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_s}_j$ representing the percentage of **sensors** for which that epoch is an outlier.


In [16]:
# This is indicator matrix C
outlier_mask = trim_ch_sd > u_out
outlier_mask

<xarray.DataArray (epoch: 277, ch: 59)> Size: 16kB
array([[ True, False,  True, ..., False, False,  True],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]], shape=(277, 59))
Coordinates:
  * epoch     (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276
  * ch        (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'
    quantile  float64 8B 0.5

In [17]:
# This is Step 2c
prop_outliers = outlier_mask.astype(float).mean('ch')
prop_outliers = prop_outliers.drop_vars('quantile')
prop_outliers  # percent of sensors that are bad during each epoch

<xarray.DataArray (epoch: 277)> Size: 2kB
array([0.06779661, 0.        , 0.        , 0.05084746, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.61016949,
       0.        , 0.01694915, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.01694915, 0.        , 0.        , 0.        ,
       0.16949153, 0.        , 0.03389831, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.25423729, 0.        ,
       0.30508475, 0.        , 0.05084746, 0.        , 0.        ,
       0.        , 0.01694915, 0.        , 0.        , 0.        ,
       0.01694915, 0.10169492, 0.08474576, 0.        , 0.18644068,
       0.47457627, 0.01694915, 0.01694915, 0.        , 0.        ,
       0.01694915, 0.        , 0.        , 0.        , 0.        ,
       0.01694915, 0.        , 0.03389831, 0.01694915, 0.01694915,
       0.        , 0.        , 0.        , 0.        , 0.01694915,
       0.        , 0.03389831, 0.01694915, 0.05084746, 0.        ,
       0.        , 0.        , 0.        , 0.01694915, 0.        ,
       0.        , 0.03389831, 0.        , 0.        , 0.03389831,
       0.        , 0.        , 0.        , 0.        , 0.        ,
...
       0.08474576, 0.        , 0.03389831, 0.01694915, 0.        ,
       0.        , 0.05084746, 0.01694915, 0.16949153, 0.        ,
       0.08474576, 0.25423729, 0.25423729, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01694915, 0.        ,
       0.03389831, 0.        , 0.01694915, 0.        , 0.        ,
       0.        , 0.        , 0.01694915, 0.01694915, 0.        ,
       0.        , 0.01694915, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.28813559, 0.74576271,
       0.33898305, 0.08474576, 0.20338983, 0.05084746, 0.        ,
       0.        , 0.01694915, 0.        , 0.10169492, 0.        ,
       0.        , 0.22033898, 0.01694915, 0.01694915, 0.        ,
       0.        , 0.        , 0.05084746, 0.        , 0.        ,
       0.        , 0.13559322, 0.06779661, 0.18644068, 0.        ,
       0.        , 0.        , 0.27118644, 0.01694915, 0.3220339 ,
       0.30508475, 0.13559322, 0.06779661, 0.18644068, 0.08474576,
       0.03389831, 0.        , 0.13559322, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.01694915, 0.01694915,
       0.        , 0.        , 0.22033898, 0.01694915, 0.03389831,
       0.6440678 , 0.16949153, 0.01694915, 0.        , 0.        ,
       0.        , 0.        ])
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 270 271 272 273 274 275 276

----

## Step 2d: Identify Noisy Epochs Part 2

Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a epoch should be marked artifactual. The epoch $j$ is flagged as noisy if $c^{\mu_s}_j > \tau^{p}$.


In [18]:
threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED EPOCHS

<xarray.DataArray (epoch: 21)> Size: 168B
array([0.61016949, 0.25423729, 0.30508475, 0.47457627, 0.38983051,
       0.27118644, 0.3559322 , 0.30508475, 0.54237288, 0.25423729,
       0.25423729, 0.28813559, 0.74576271, 0.33898305, 0.20338983,
       0.22033898, 0.27118644, 0.3220339 , 0.30508475, 0.22033898,
       0.6440678 ])
Coordinates:
  * epoch    (epoch) int64 168B 14 43 45 60 100 113 ... 231 247 249 250 267 270

In [19]:
# Let's add the outlier epochs to our flags
bad_epochs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED Epochs
pipeline.flags["epoch"].add_flag_cat(kind='ch_sd',
                                  bad_epoch_inds=bad_epochs,
                                     epochs=epochs)
print(pipeline.flags['epoch'])

📋 LOSSLESS: 20.98 second(s) flagged as BAD_LL_ch_sd
Flagged epochs: |
  Noisy: None
  Noisy Ics: None
  Uncorrelated: None



------


<br>
<br>


# **Step 3**: Find Nearest Neighbours & return Maximum Correlation


*Wheras steps 1 and 2 operated on a 2D matrix of standard deviation values, Steps 5, 6, 7, and 8 will operate on correlation coefficients. This Step describes the procedure for defining the 2D matrix of correlation coefficients.*

<br>

![Get nearest neighbours](https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Nearest_neighbors.png)


-----

In [20]:
from pylossless.pipeline import chan_neighbour_r
# Posting full function definition for easier inspection
chan_neighbour_r??

Signature: chan_neighbour_r(epochs, nneigbr, method)
Source:   
def chan_neighbour_r(epochs, nneigbr, method):
    """Compute nearest Neighbor R.

    Parameters
    ----------
    epochs : mne.Epochs

    nneigbr : int
        Number of neighbours to compare in open interval

    method : str
        One of 'max', 'mean', or 'trimmean'. This is the function
        which aggregates the neighbours into one value.

    Returns
    -------
    Xarray : Xarray.DataArray
        An instance of Xarray.DataArray
    """
    chan_locs = pd.DataFrame(epochs.get_montage().get_positions()["ch_pos"]).T
    chan_dist = pd.DataFrame(
        distance_matrix(chan_locs, chan_locs),
        columns=chan_locs.index,
        index=chan_locs.index,
    )
    rank = chan_dist.rank("columns", ascending=True) - 1
    rank[rank == 0] = np.nan
    nearest_neighbor = pd.DataFrame(
        {
            ch_name: row.dropna().sort_values()[:nneigbr].index.values
            for ch_name, row in rank.iterrows()
  

<br>

#### **Take a moment to notice** that the flagged epochs from the previous step are dropped during epoching.

<br>

In [21]:
# Notice that our flagged epochs are dropped.
epochs = pipeline.get_epochs()

🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 601 original time points ...
34 bad epochs dropped
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.



----------------------



## Step 3, part 1: Calculate Correlation Coefficents between each Sensor and it's nearest Neighbors

<br>

- For each good sensor $i$ in $S_{\mathcal{G}}$, we select its $N$ nearest neighbors. I.e. the $N$ sensors that are closest to it.

- We call the sensor $i$ the *origin*, and its nearest neighbors $\hat{s_l}$ with $l \in \{1, 2, \ldots, N\}$

- Then, for each epoch $j$, we calculate the correlation coefficient $\rho^t_{(i,\hat{s_l}),j}$ between origin sensor $i$ and each neighbor $\hat{s_l}$ across dimension $t$ (samples):

Returning a 3D matrice of correlation coefficients

$$
\mathrm{P}^t = \{\rho^t_{(i, \hat{s_l}),j}\} \in \mathbb{R}^{S_G \times E_G \times n}
$$

Finally, we select the maximum correlation coefficient across the neighbor dimension $n$:


$$
\mathrm{P}^{t,{\text{max}}^n}= \max\limits_{\hat{s_l}}  \rho^t_{(i, \hat{s_l}),j}
$$


Returning a 2D matrix where each value at $(i, j)$ is the maximum correlation coefficient between sensor $i$ and its $N$ nearest neighbors, for epoch $j$

------------------------

In [22]:
data_r_ch = chan_neighbour_r(epochs, nneigbr=3, method='max')
# maximum correlation out of correlations between ch and its 3 neighbors
data_r_ch

100%|██████████| 59/59 [00:02<00:00, 21.11it/s]


<xarray.DataArray (ch: 59, epoch: 243)> Size: 115kB
array([[0.92446759, 0.57336808, 0.91130302, ..., 0.76069933, 0.73013373,
        0.69100602],
       [0.97271908, 0.69150223, 0.96271193, ..., 0.84921217, 0.8365151 ,
        0.8927969 ],
       [0.97271908, 0.69150223, 0.96271193, ..., 0.84921217, 0.8365151 ,
        0.8927969 ],
       ...,
       [0.98408672, 0.90868413, 0.98275535, ..., 0.91041092, 0.88193036,
        0.92706497],
       [0.97481915, 0.84343495, 0.95507698, ..., 0.78112982, 0.92466046,
        0.83560937],
       [0.93824729, 0.74132503, 0.90936736, ..., 0.66620867, 0.71736872,
        0.80682815]], shape=(59, 243))
Coordinates:
  * epoch    (epoch) int64 2kB 0 1 2 3 4 5 6 7 ... 236 237 238 239 240 241 242
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'


---------------------

<br>

### This matrix $\mathrm{P}^{t,{\text{max}}^n}$  will be used in steps 5, 6, 7, and 8 below.

<br>

-------------------

# **Step 5**: Flag Bridged Sensors

<br>


<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/Flag_bridged_sensors.png" width="50%" />



<br>

---------------------

## <center>Operations<center>

<br>

### 6a) Take the 25th, 50th, and 75th quantile across the epochs dimension of $\mathrm{P}^{t,{\text{max}}^n}$

#### *and calculate the Inter Quantile Range (IQR)*


$$
IQR^e = \mathrm{P}^{t,{\text{max}}^nQ75^e} - \mathrm{P}^{t,{\text{max}}^nQ25^e}
$$

For each sensor, divide the median across epochs by the IQR across epochs. bridged channels should have a high median correlation but a low IQR of the correlation. We call this measure the bridge-indicator.

$$
\mathcal{B}_s = \frac{\mathrm{P}^{t,{\text{max}}^nQ50^e}}{IQR^e}
$$

<br>

-------------------------------



### 6b) Define a bridging threshold

Take the 25th, 50th, and 75th quantile of $\mathcal{B}_s$

And calculate the Inter Quantile Range* $IQR^s$. A channel $i$ is bridged if $\mathcal{B}_i > B^{Q50^s} +k \times IQR^s$.

In [23]:
import scipy
from functools import partial

msr = data_r_ch.median("epoch") / data_r_ch.reduce(scipy.stats.iqr, dim="epoch")
# msr is a 1D vector of size n_sensors
config_trim = 40
config_bridge_z = 6

trim = config_trim
if trim >= 1:
        trim /= 100
trim /= 2 # .20 and will be used as (.20, .20)

trim_mean = partial(scipy.stats.mstats.trimmed_mean,
                        limits=(trim, trim))
trim_std = partial(scipy.stats.mstats.trimmed_std,
                        limits=(trim, trim))

z_val = config_bridge_z  # 6
mask = (msr > msr.reduce(trim_mean, dim="ch")
        + z_val*msr.reduce(trim_std, dim="ch")) # bridged chans

bridged_ch_names = data_r_ch.ch.values[mask]
bridged_ch_names

array(['EEG 012', 'EEG 013', 'EEG 054', 'EEG 057'], dtype='<U7')

In [24]:
# Let's add the outlier channels to our flags
bad_chs = bridged_ch_names
pipeline.flags["ch"].add_flag_cat(kind='bridge',
                                  bad_ch_names=bad_chs)
pipeline.flags['ch']

Flagged channels: |
  Noisy: None
  Bridged: None
  Uncorrelated: None
  Rank: None

# Step 6: Flag Rank Channel

## The rank sensor $i$ is simply the sensor with the highest median (across epochs) correlation coefficient, out of the remaining set of good sensors, i.e.,

$$
i = \text{arg}\max\limits_i \rho_{j}^{t,{\text{max}}^n,median^j}
$$

In [25]:
good_chs = [ch for ch in data_r_ch.ch.values
          if ch not in pipeline.flags["ch"].get_flagged()]
data_r_ch_good = data_r_ch.sel(ch=good_chs)

flag_ch = [str(data_r_ch_good.median("epoch")
                                     .idxmax(dim="ch")
                                     .to_numpy()
                            )]
pipeline.flags['ch'].add_flag_cat(kind='rank', bad_ch_names=flag_ch)
pipeline.flags['ch']['rank']

['EEG 058']


--------------------------------------

# Step 7: Flag low correlation Channels

- This step closely follows Step 1 except we use the lower end of the distribution to set the outliers threshold.

- We will be succinct in the documentation. Please Feel free to inspect the variables in the python code as needed:


----------------

## <center>Operations<center>

<br>

### 6a) Take the 30th and 50th quantile of $\mathrm{P}^{t,{\text{max}}^n}$ across the sensors dimension

*and calculate the Lower Quantile Range* $LQR^s$

Resulting in vectors $\mathrm{P}^{t,{\text{max}}^nQ25^s}$ and $\mathrm{P}^{t,{\text{max}}^nQ50^s}$ of size $E_\mathcal{G}$. The lower quantile range ($LQR$) is defined as:


$$
LQR^s = \mathrm{P}^{t,{\text{max}}^nQ50^s} - \mathrm{P}^{t,{\text{max}}^nQ25^s}
$$

<br>

### 6b) Define epoch-specific thresholds for rejecting sensors

$\tau^s_j$ represents our set of epoch-specific thresholds:


$$
\tau^s = \mathrm{P}^{t,{\text{max}}^nQ50^s} - LQR^s\times k
$$

Now, we compare our 2D correlation coefficient matrix $\mathrm{P}^{t,{\text{max}}^n} = \{ \rho^{t,{\text{max}}^n}_{ij} \}$ to the threshold vector $\tau^s_j$ resulting in the indicator matrix $C \in \{0, 1\}^{S \times E}=\{c_{ij}\}$ with

$$
c_{ij} =
\begin{cases}
1 & \text{if } \rho^{t,{\text{max}}^n}_{ij} \leq \tau^s_j \\
0 & \text{if } \rho^{t,{\text{max}}^n}_{ij} > \tau^s_j
\end{cases}
$$



### 6c) Identify uncorrelated sensors

To identify uncorrelated **sensors**, we average across the **epoch** dimension of our indicator matrix $C$ and obtain $C^{\mu_e} \in \mathbb{R}^{S_\mathcal{G}}$, which is a vector of fractional numbers $c^{\mu_e}_i$ representing the percentage of **epochs** for which that sensor is an outlier.


Next, we define a fractional threshold $\tau^{p}$ ($p$ for percentile; default, ``.20``) as a cutoff point for determining if a sensor should be marked artifactual. The sensor $i$ is flagged as noisy if $c^{\mu_e}_i > \tau^{p}$.

In [26]:
# Step a
lower_val, mid_val = data_r_ch.quantile([.3, .5], dim='ch')

# step B. Define the threshold
lower_dist = mid_val - lower_val

# Step B
k = 3
l_out =  mid_val - lower_dist * k

# Step D
outlier_mask = data_r_ch < l_out

prop_outliers = outlier_mask.astype(float).mean('epoch')
prop_outliers = prop_outliers.drop_vars('quantile')

threshold = 0.2
prop_outliers[prop_outliers > threshold] # FLAGGED SENSORS

# Let's add the outlier channels to our flags
bad_chs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED SENSORS
pipeline.flags["ch"].add_flag_cat(kind='uncorrelated',
                                  bad_ch_names=bad_chs)
print(pipeline.flags['ch'])

Flagged channels: |
  Noisy: None
  Bridged: None
  Uncorrelated: ['EEG 004' 'EEG 005' 'EEG 008' 'EEG 015' 'EEG 016' 'EEG 018' 'EEG 019'
 'EEG 023']
  Rank: ['EEG 058']




----------------------------

<br>

# Step 8: Flag Uncorrelated Epochs

This step is similar to step 6 (flag uncorrelated channels) and step 2 (flag noisy epochs). For the sake of brevity we will not re-describe each step


---------------------------------

## <center>Operations<center>

<br>

### 6a) Take $LQR^e$ of $\mathrm{P}^{t,{\text{max}}^n}$


### 6b) Defined sensor-specific thresholds

We get the indicator matrix as described previously, using

$$
 \tau^e_i = \mathrm{P}^{t,{\text{max}}^nQ50^e} - LQR^e\times k
$$

and

$$
c_{ij} =
\begin{cases}
1 & \text{if } \rho^{t,{\text{max}}^n}_{ij} < \tau^e_i \\
0 & \text{if } \rho^{t,{\text{max}}^n}_{ij} \geq \tau^e_i
\end{cases}
$$


<br>

### 6c) Identify uncorrelated sensors
<br>

We define a fractional threshold as we did in the previous step and flag uncorrelated epochs $j$ if $c^{\mu_s}_j > \tau^{p}$.



In [27]:
# Step a
lower_val, mid_val = data_r_ch.quantile([.3, .5], dim='epoch')

# step B. Define the threshold
lower_dist = mid_val - lower_val

# Step B
k = 3
l_out =  mid_val - lower_dist * k

# Step D
outlier_mask = data_r_ch < l_out

prop_outliers = outlier_mask.astype(float).mean('ch')
prop_outliers = prop_outliers.drop_vars('quantile')

threshold = 0.2
prop_outliers[prop_outliers < threshold] # FLAGGED EPOCHS

# Let's add the outlier epochs to our flags
bad_epochs = prop_outliers[prop_outliers > threshold].coords.to_index().values # FLAGGED EPOCHS
pipeline.flags["epoch"].add_flag_cat(kind='uncorrelated',
                                  bad_epoch_inds=bad_epochs,
                                     epochs=epochs)
print(pipeline.flags['ch'])

📋 LOSSLESS: 8.99 second(s) flagged as BAD_LL_uncorrelated
Flagged channels: |
  Noisy: None
  Bridged: None
  Uncorrelated: ['EEG 004' 'EEG 005' 'EEG 008' 'EEG 015' 'EEG 016' 'EEG 018' 'EEG 019'
 'EEG 023']
  Rank: ['EEG 058']



# Step 9: Run Initial ICA

In [28]:
pipeline.run_ica('run1')

LOSSLESS: 👇 run_ica.
🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 601 original time points ...
50 bad epochs dropped
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Fitting ICA to data using 46 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 45 components
Fitting ICA took 3.1s.
LOSSLESS: 🏁 Finished run_ica after 4.39 seconds.


# Step 10: Identify outlying ICA Time courses

This step follows exactly step 1 so we will not re-describe the operations here.

There are however a few key differences:

- The input 3D matrix is ``(n_components, n_good_epochs, n_times)``: $ X \in \mathbb{R}^{C\times E_G \times{T}} $
- the default threshold $k$ is 6

In [29]:
pipeline.flags["ic"]

""


# Step 11: Run Final ICA

# This is the Final ICA, excluding all flagged epochs and channels.

# The ``ICLabel`` Neural Network will be run on this ICA

In [30]:
pipeline.run_ica('run2')

LOSSLESS: 👇 run_ica.
🧹 Epoching..
Not setting metadata
277 matching events found
No baseline correction applied
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Using data from preloaded Raw for 277 events and 601 original time points ...
50 bad epochs dropped
🔍 Detecting channels to leave out of reference.
EEG channel type selected for re-referencing
Applying a custom ('EEG',) reference.
Fitting ICA to data using 46 channels (please be patient, this may take a while)
Selecting by non-zero PCA components: 45 components
Computing Extended Infomax ICA
Fitting ICA took 25.7s.


/Users/scotterik/miniforge3/envs/lossless/lib/python3.10/site-packages/mne_icalabel/iclabel/features.py:67: RuntimeWarning: The provided Epochs instance is not filtered between 1 and 100 Hz. ICLabel was designed to classify features extracted from an EEG dataset bandpass filtered between 1 and 100 Hz (see the 'filter()' method for Raw and Epochs instances).
  warn(


LOSSLESS: 🏁 Finished run_ica after 31.06 seconds.


# Appendix: Find outlying channels to leave out of Re Reference

- This robust average reference is performed before the first pipeline step and between most pipeline steps.

- It does not flag sensors but identifies outlier sensors and leaves them out of the reference

Like steps 1 and 2, this step starts with 3D matrix $X \in \mathbb{R}^{S \times E \times T}$ and takes the standard deviation across axis $T$:

<br>

<img src="https://raw.githubusercontent.com/scott-huberty/wip_pipeline-figures/main/robust_rereference.png" alt="Find outliers and leave out of rereference" width="50%" height="50%">


<br>

---------------------

## <center>Operations<center>

<br>

### 6a) Take the median, 30th, and 70th quantile across dimension channels of  $X^{\sigma_{t}} \in \mathbb{R}^{S \times E}$, and Calculate the Inter Quantile Range (IQR).

<br>


Create a non-parametric Z score

$$
\mathcal{Z}^{\sigma^{t}} = \frac{X^{\sigma^{t}Q50^s} - X^{\sigma^{t}}}{IQR^s}
$$




<br>

-------------------------------

<br>


### 6b) Define  threshold

Sensor $i$ is left out of the re-reference if

$$
\mathcal{Z}^{\sigma^{t}\mu^{e}} > \mathcal{Z}^{\sigma^{t}\mu^{e}Q50^s} + k \times IQR^s
$$

with $IQR^s = \mathcal{Z}^{\sigma^{t}\mu^{e}Q75^s}-\mathcal{Z}^{\sigma^{t}\mu^{e}Q25^s}$ and where $\mathcal{Z}^{\sigma^{t}\mu^{e}}$ is the mean of $\mathcal{Z}^{\sigma^{t}}$ across epochs.

In [31]:
# This is step 2a
ch_dist = trim_ch_sd - trim_ch_sd.median(dim="ch")
perc_30 = trim_ch_sd.quantile(0.3, dim="ch")
perc_70 = trim_ch_sd.quantile(0.7, dim="ch")

# This is step b: non-parametric z-score
ch_dist /= perc_70 - perc_30  # shape (chans, epoch)

# This is step c
mean_ch_dist = ch_dist.mean(dim="epoch")

# Step D begins here
# find the median and 30 and 70 percentiles
# of the mean of the channel distributions
mdn = np.median(mean_ch_dist)
deviation = np.diff(np.quantile(mean_ch_dist, [0.3, 0.7]))
print(f"median: {mdn}, std: {deviation}")

# Thresholding. True sensors are left out
mean_ch_dist > mdn+6*deviation

median: 0.012408492191093343, std: [0.73812015]


<xarray.DataArray (ch: 59)> Size: 59B
array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False])
Coordinates:
  * ch       (ch) <U7 2kB 'EEG 001' 'EEG 002' 'EEG 003' ... 'EEG 059' 'EEG 060'

In [32]:
# Here are any sensor names that are excluded
mean_ch_dist.ch[mean_ch_dist > mdn+6*deviation].values.tolist()

[]